# VacationPy
---

## Starter Code to Import Libraries and Load the Weather and Coordinates Data

In [1]:
# Dependencies and Setup
import hvplot.pandas
import pandas as pd
import requests

# Import API key
from api_keys import geoapify_key

In [2]:
# Load the CSV file created in Part 1 into a Pandas DataFrame
city_data_df = pd.read_csv("output_data/cities.csv")

# Display sample data
city_data_df.head()

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,0,lionel town,17.8101,-77.2406,84.07,61,36,14.32,JM,1738340653
1,1,yanam,16.7333,82.2167,67.84,98,18,0.40,IN,1738340655
2,2,puerto natales,-51.7236,-72.4875,63.05,36,75,29.93,CL,1738340361
3,3,georgetown,5.4112,100.3354,82.33,56,20,9.22,MY,1738340379
4,4,lewistown,40.5992,-77.5714,36.61,100,100,0.00,US,1738340390


---

### Step 1: Create a map that displays a point for every city in the `city_data_df` DataFrame. The size of the point should be the humidity in each city.

In [27]:
#%%capture --no-display

# Configure the map plot
city_data_map = city_data_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
)

# Display the map
city_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity)

### Step 2: Narrow down the `city_data_df` DataFrame to find your ideal weather condition

In [35]:
# Narrow down cities that fit criteria and drop any results with null values
ideal_temp_df = city_data_df.loc[(city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80) & (city_data_df["Cloudiness"] < 30) & (city_data_df["Wind Speed"] < 15),:]
#city_data_df[(city_data_df["Max Temp"] > 70) & (city_data_df["Max Temp"] < 80)]
# Drop any rows with null values
ideal_temp_df.dropna()

# Display sample data
ideal_temp_df

,City_ID,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
151,151,alice springs,-23.7000,133.8833,71.15,23,0,1.14,AU,1738340847
153,153,port alfred,-33.5906,26.8910,71.19,84,28,5.01,ZA,1738340849
185,185,valparaiso,-33.0393,-71.6273,73.47,55,0,13.80,CL,1738340668
216,216,broken hill,-31.9500,141.4333,71.85,24,0,11.90,AU,1738340932
228,228,woolgoolga,-30.1106,153.2007,71.94,88,27,10.33,AU,1738340948
236,236,bamboo flat,11.7000,92.7167,77.20,74,9,9.15,IN,1738340958
237,237,perth,-31.9333,115.8333,72.03,88,0,5.75,AU,1738340959
262,262,talcahuano,-36.7167,-73.1167,73.08,64,20,14.97,CL,1738340716
311,311,hermanus,-34.4187,19.2345,70.81,68,0,11.72,ZA,1738341057
352,352,howrah,22.5892,88.3103,75.20,88,20,5.75,IN,1738341110


### Step 3: Create a new DataFrame called `hotel_df`.

In [36]:
# Use the Pandas copy function to create DataFrame called hotel_df to store the city, country, coordinates, and humidity
hotel_df = ideal_temp_df[['City', 'Country', 'Lat', 'Lng', 'Humidity']].copy()

# Add an empty column, "Hotel Name," to the DataFrame so you can store the hotel found using the Geoapify API
hotel_df['Hotel Name'] = ''

# Display sample data
hotel_df

,City,Country,Lat,Lng,Humidity,Hotel Name
151,alice springs,AU,-23.7000,133.8833,23,
153,port alfred,ZA,-33.5906,26.8910,84,
185,valparaiso,CL,-33.0393,-71.6273,55,
216,broken hill,AU,-31.9500,141.4333,24,
228,woolgoolga,AU,-30.1106,153.2007,88,
236,bamboo flat,IN,11.7000,92.7167,74,
237,perth,AU,-31.9333,115.8333,88,
262,talcahuano,CL,-36.7167,-73.1167,64,
311,hermanus,ZA,-34.4187,19.2345,68,
352,howrah,IN,22.5892,88.3103,88,


### Step 4: For each city, use the Geoapify API to find the first hotel located within 10,000 metres of your coordinates.

In [37]:
# Set parameters to search for a hotel
radius = 10000
params = {"categories":"accommodation.hotel",
    "apiKey" : geoapify_key,
    "format":"json"
}

# Print a message to follow up the hotel search
print("Starting hotel search")

# Iterate through the hotel_df DataFrame
for index, row in hotel_df.iterrows():
    # get latitude, longitude from the DataFrame
    latitude = row["Lat"]
    longitude = row["Lng"]

    # Add the current city's latitude and longitude to the params dictionary
    params["filter"] = f"circle:{longitude},{latitude},{radius}"
    params["bias"] = f"proximity:{longitude},{latitude}"

    # Set base URL
    base_url = "https://api.geoapify.com/v2/places"

    # Make and API request using the params dictionary
    name_address = requests.get(base_url, params=params)

    # Convert the API response to JSON format
    name_address = name_address.json()

    # Grab the first hotel from the results and store the name in the hotel_df DataFrame
    try:
        hotel_df.loc[index, "Hotel Name"] = name_address["features"][0]["properties"]["name"]
    except (KeyError, IndexError):
        # If no hotel is found, set the hotel name as "No hotel found".
        hotel_df.loc[index, "Hotel Name"] = "No hotel found"

    # Log the search results
    print(f"{hotel_df.loc[index, 'City']} - nearest hotel: {hotel_df.loc[index, 'Hotel Name']}")

# Display sample data
hotel_df

Starting hotel search
alice springs - nearest hotel: Aurora Alice Springs
port alfred - nearest hotel: No hotel found
valparaiso - nearest hotel: Ibis Hotel
broken hill - nearest hotel: Ibis Styles
woolgoolga - nearest hotel: Darlington Beach Resort
bamboo flat - nearest hotel: Hotel Megapode Nest
perth - nearest hotel: Oxford Hotel
talcahuano - nearest hotel: Club Recreativo Armada de Chile
hermanus - nearest hotel: Aloe guest house
howrah - nearest hotel: Sun India Guest House
abepura - nearest hotel: Yoyo
banda aceh - nearest hotel: Hotel 61
inhambane - nearest hotel: Hotel de Inhambane
jolo - nearest hotel: No hotel found
goondiwindi - nearest hotel: Queensland Hotel
port blair - nearest hotel: Andaman Ocean Breeze
polavaram - nearest hotel: Rajugari hotel
ingeniero jacobacci - nearest hotel: Gran Hotel Argentino
cosala - nearest hotel: Hotel Quinta Minera


,City,Country,Lat,Lng,Humidity,Hotel Name
151,alice springs,AU,-23.7000,133.8833,23,Aurora Alice Springs
153,port alfred,ZA,-33.5906,26.8910,84,No hotel found
185,valparaiso,CL,-33.0393,-71.6273,55,Ibis Hotel
216,broken hill,AU,-31.9500,141.4333,24,Ibis Styles
228,woolgoolga,AU,-30.1106,153.2007,88,Darlington Beach Resort
236,bamboo flat,IN,11.7000,92.7167,74,Hotel Megapode Nest
237,perth,AU,-31.9333,115.8333,88,Oxford Hotel
262,talcahuano,CL,-36.7167,-73.1167,64,Club Recreativo Armada de Chile
311,hermanus,ZA,-34.4187,19.2345,68,Aloe guest house
352,howrah,IN,22.5892,88.3103,88,Sun India Guest House


### Step 5: Add the hotel name and the country as additional information in the hover message for each city in the map.

In [38]:
#%%capture --no-display

# Configure the map plot
ideal_data_map = hotel_df.hvplot.points(
    "Lng",
    "Lat",
    geo = True,
    tiles = "OSM",
    frame_width = 800,
    frame_height = 600,
    size = "Humidity",
    scale = 1,
    color = "City",
    hover_cols = ["Country", "Hotel Name"]
)

# Display the map
ideal_data_map

:Overlay
   .WMTS.I   :WMTS   [Longitude,Latitude]
   .Points.I :Points   [Lng,Lat]   (City,Humidity,Country,Hotel Name)